# Import Libraries

In [ ]:
!pip install opencv-python==4.5.5.62
!pip install opencv-contrib-python==4.5.5.62
!pip install tf-explain

In [ ]:
import os
import gc
import sys

import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import skimage
from skimage.feature import hog, canny
from skimage.filters import sobel
from skimage import color

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from keras import layers
import keras.backend as K
from keras.models import Sequential, Model
from keras.preprocessing import image
from keras.layers import Input, Dense, Activation, Dropout
from keras.layers import Flatten, BatchNormalization
from keras.layers import Convolution2D, MaxPooling2D, AveragePooling2D, GlobalAveragePooling2D 
from keras.applications.imagenet_utils import preprocess_input
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications import ResNet50
from tf_explain.core.activations import ExtractActivations
from tf_explain.core.grad_cam import GradCAM
from sklearn.model_selection import train_test_split
from keras.utils.data_utils import get_file

from PIL import Image
from tqdm import tqdm
import random as rnd
import cv2
from keras.preprocessing.image import ImageDataGenerator
from numpy import expand_dims

!pip install livelossplot
from livelossplot import PlotLossesKeras

%matplotlib inline

# Loading Dataset
We'll use here the Pandas to load the dataset into memory

In [ ]:
train_df = pd.read_csv('../input/state-farm-distracted-driver-detection/driver_imgs_list.csv')
train_df['path'] = '../input/state-farm-distracted-driver-detection/imgs/train/' + train_df['classname']+ '/' +train_df['img']
pred_df = pd.read_csv('../input/state-farm-distracted-driver-detection/sample_submission.csv')

In [ ]:
classes = {'c0': 'normal driving',
'c1': 'texting - right',
'c2': 'talking on the phone - right',
'c3': 'texting - left',
'c4': 'talking on the phone - left',
'c5': 'operating the radio',
'c6': 'drinking',
'c7': 'reaching behind',
'c8': 'hair and makeup',
'c9': 'talking to passenger',}

In [ ]:
train_df.head(10)

In [ ]:
pred_df.head()

In [ ]:
train_df.count()

In [ ]:
print('Train samples count: ', len(train_df))
train_df.columns

In [ ]:
print('Class Count: ',len(train_df['classname'].value_counts()))
train_df['classname'].value_counts()

# Checking missing data
Lets check if there is any missing values in our dataset

In [ ]:
train_df.isna().sum()

# Visualization
Looking at some random beauties
It's a great deal of fun to explore the data and play around with matplotlib

In [ ]:
plt.figure(figsize = (15,12))
for idx,i in enumerate(train_df.classname.unique()):
    plt.subplot(4,7,idx+1)
    df = train_df[train_df['classname'] ==i].reset_index(drop = True)
    image_path = df.loc[rnd.randint(0, len(df))-1,'path']
    img = Image.open(image_path)
    img = img.resize((224,224))
    plt.imshow(img)
    plt.axis('off')
    plt.title(classes[i])
plt.tight_layout()
plt.show()

In [ ]:
def plot_species(df,class_name):
    plt.figure(figsize = (12,12))
    classes_df = train_df[train_df['classname'] ==  class_name].reset_index(drop = True)
    plt.suptitle(classes[class_name])
    for idx,i in enumerate(np.random.choice(classes_df['path'],32)):
        plt.subplot(8,8,idx+1)
        image_path = i
        img = Image.open(image_path)
        img = img.resize((224,224))
        plt.imshow(img)
        plt.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
for class_ in train_df['classname'].unique():
    #print('\n\n')
    plot_species(train_df , class_)

# Class Distribution Analysis¶
In this section we will be analyzing the number of training and test samples in each class. It will give us a better understanding of our dataset and provide us the necessary information to preprocess our dataset before the training phase.

In [ ]:
plot = sns.countplot(x = train_df['classname'], color = '#2596be')
sns.set(rc={'figure.figsize':(30,25)})
sns.despine()
plot.set_title('Class Distribution\n', font = 'serif', x = 0.1, y=1, fontsize = 18);
plot.set_ylabel("Count", x = 0.02, font = 'serif', fontsize = 12)
plot.set_xlabel("Driver classes", fontsize = 15, font = 'serif')

for p in plot.patches:
    plot.annotate(format(p.get_height(), '.0f'), (p.get_x() + p.get_width() / 2, p.get_height()), 
       ha = 'center', va = 'center', xytext = (0, -20),font = 'serif', textcoords = 'offset points', size = 15)

In [ ]:
plt.figure(figsize=(5,5))
class_cnt = train_df.groupby(['classname']).size().reset_index(name = 'counts')
colors = sns.color_palette('Paired')[0:9]
plt.pie(class_cnt['counts'], labels=class_cnt['classname'], colors=colors, autopct='%1.1f%%')
plt.legend(loc='upper right')
plt.show()

# Image Resolutions

In [ ]:
widths, heights = [], []

for path in tqdm(train_df["path"]):
    width, height = Image.open(path).size
    widths.append(width)
    heights.append(height)
    
train_df["width"] = widths
train_df["height"] = heights
train_df["dimension"] = train_df["width"] * train_df["height"]

**Lets see some small images**

In [ ]:
train_df.sort_values('width').head(84)

# Color Analysis
We need to do some color analysis to get an ida about the augmentation technique needed for this problem

In [ ]:
def is_grey_scale(givenImage):
    w,h = givenImage.size
    for i in range(w):
        for j in range(h):
            r,g,b = givenImage.getpixel((i,j))
            if r != g != b: return False
    return True

**Check color scale of Train images**

In [ ]:
sampleFrac = 0.5
#get our sampled images
isGreyList = []
for imageName in train_df['path'].sample(frac=sampleFrac):
    val = Image.open(imageName).convert('RGB')
    isGreyList.append(is_grey_scale(val))
print(np.sum(isGreyList) / len(isGreyList))
del isGreyList

**Get mean intensity for each channel RGB**

In [ ]:
def get_rgb_men(row):
    img = cv2.imread(row['path'])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return np.sum(img[:,:,0]), np.sum(img[:,:,1]), np.sum(img[:,:,2])

tqdm.pandas()
train_df['R'], train_df['G'], train_df['B'] = zip(*train_df.progress_apply(lambda row: get_rgb_men(row), axis=1))

In [ ]:
def show_color_dist(df, count):
    fig, axr = plt.subplots(count,2,figsize=(15,15))
    if df.empty:
        print("Image internsity of selected color is weak")
        return
    for idx, i in enumerate(np.random.choice(df['path'], count)):
        img = cv2.imread(i)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        axr[idx,0].imshow(img)
        axr[idx,0].axis('off')
        axr[idx,1].set_title('R={:.0f}, G={:.0f}, B={:.0f} '.format(np.mean(img[:,:,0]), np.mean(img[:,:,1]), np.mean(img[:,:,2]))) 
        x, y = np.histogram(img[:,:,0], bins=255)
        axr[idx,1].bar(y[:-1], x, label='R', alpha=0.8, color='red')
        x, y = np.histogram(img[:,:,1], bins=255)
        axr[idx,1].bar(y[:-1], x, label='G', alpha=0.8, color='green')
        x, y = np.histogram(img[:,:,2], bins=255)
        axr[idx,1].bar(y[:-1], x, label='B', alpha=0.8, color='blue')
        axr[idx,1].legend()
        axr[idx,1].axis('off')

### Red images and their color distribution
Since we are picking random images, some image may appear multiple times

In [ ]:
df = train_df[((train_df['B']) < train_df['R']) & ((train_df['G']) < train_df['R'])]
if df.size != 0:
    show_color_dist(df, 8)

### Green images and their color distribution
Since we are picking random images, some image may appear multiple times

In [ ]:
df = train_df[(train_df['G'] > train_df['R']) & (train_df['G'] > train_df['B'])]
if df.size != 0:
    show_color_dist(df, 8)

### Blue images and their color distribution
Since we are picking random images, some image may appear multiple times

In [ ]:
df = train_df[(train_df['B'] > train_df['R']) & (train_df['B'] > train_df['G'])]
if df.size != 0:
    show_color_dist(df, 8)

# Features

## Analyzing Edges
A Sobel filter is one means of getting a basic edge magnitude/gradient image. Can be useful to threshold and find prominent linear features, etc. Several other similar filters in skimage.filters are also good edge detectors: roberts, scharr, etc. and you can control direction, i.e. use an anisotropic version.

In [ ]:
def edges_images_gray(class_name):
    classes_df = train_df[train_df['classname'] ==  class_name].reset_index(drop = True)
    for idx,i in enumerate(np.random.choice(classes_df['path'],2)):
        image = cv2.imread(i)
        gray=cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        edges = sobel(image)
        gray_edges=sobel(gray)
        dimension = edges.shape
        fig = plt.figure(figsize=(8, 8))
        plt.suptitle(classes[class_name])
        plt.subplot(2,2,1)
        plt.imshow(gray_edges)
        plt.subplot(2,2,2)
        plt.imshow(edges[:dimension[0],:dimension[1],0], cmap="gray")
        plt.subplot(2,2,3)
        plt.imshow(edges[:dimension[0],:dimension[1],1], cmap='gray')
        plt.subplot(2,2,4)
        plt.imshow(edges[:dimension[0],:dimension[1],2], cmap='gray')
        plt.show()

In [ ]:
for class_name in train_df['classname'].unique():
    edges_images_gray(class_name)

# HSV Transform
Since this contest is about time series ordering, I think it's possible there may be useful information in a transform to HSV color space. HSV is useful for identifying shadows and illumination, as well as giving us a means to identify similar objects that are distinct by color between scenes (hue), though there's no guarantee the hue will be stable.

In [ ]:
def hsv_images(class_name):
    classes_df = train_df[train_df['classname'] ==  class_name].reset_index(drop = True)
    for idx,i in enumerate(np.random.choice(train_df['path'],2)):  
        image = cv2.imread(i)
        hsv = color.rgb2hsv(image)
        dimension = hsv.shape
        fig = plt.figure(figsize=(8, 8))
        plt.suptitle(classes[class_name])
        plt.subplot(2,2,1)
        plt.imshow(image)
        plt.subplot(2,2,2)
        plt.imshow(hsv[:dimension[0],:dimension[1],0], cmap="PuBuGn")
        plt.subplot(2,2,3)
        plt.imshow(hsv[:dimension[0],:dimension[1],1], cmap='PuBuGn')
        plt.subplot(2,2,4)
        plt.imshow(hsv[:dimension[0],:dimension[1],2], cmap='PuBuGn')
        plt.show()

In [ ]:
for class_name in train_df['classname'].unique():
    hsv_images(class_name)

## Corners

In [ ]:
def corners_images_gray(class_name):
    classes_df = train_df[train_df['classname'] ==  class_name].reset_index(drop = True)
    for idx,i in enumerate(np.random.choice(classes_df['path'],4)):
        image = cv2.imread(i)
        gray=cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        corners_gray = cv2.goodFeaturesToTrack(gray, maxCorners=50, qualityLevel=0.02, minDistance=20)
        corners_gray = np.float32(corners_gray)
        for item in corners_gray:
            x, y = item[0]
            cv2.circle(image, (int(x), int(y)), 6, (0, 255, 0), -1)
        fig = plt.figure(figsize=(16, 16))
        plt.suptitle(classes[class_name])
        plt.subplot(2,2,1)
        plt.imshow(image, cmap="BuGn")
        plt.show()

In [ ]:
for class_name in train_df['classname'].unique():
    corners_images_gray(class_name)

## Sift Features

In [ ]:
def sift_images_gray(class_name):
    classes_df = train_df[train_df['classname'] ==  class_name].reset_index(drop = True)
    for idx,i in enumerate(np.random.choice(classes_df['path'],4)):
        image = cv2.imread(i)
        gray=cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        sift = cv2.SIFT_create()
        kp, des = sift.detectAndCompute(gray, None)
        kp_img = cv2.drawKeypoints(image, kp, None, color=(0, 255, 0), flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
        fig = plt.figure(figsize=(16, 16))
        plt.suptitle(classes[class_name])
        plt.subplot(2,2,1)
        plt.imshow(kp_img, cmap="viridis")
        plt.show()

In [ ]:
for class_name in train_df['classname'].unique():
    sift_images_gray(class_name)

# Plot Augmentations

In [ ]:
def plot_augimages(paths, datagen):
    plt.figure(figsize = (14,28))
    plt.suptitle('Augmented Images')
    
    midx = 0
    for path in paths:
        data = Image.open(path)
        data = data.resize((224,224))
        samples = expand_dims(data, 0)
        it = datagen.flow(samples, batch_size=1)
    
        # Show Original Image
        plt.subplot(10,5, midx+1)
        plt.imshow(data)
        plt.axis('off')
    
        # Show Augmented Images
        for idx, i in enumerate(range(4)):
            midx += 1
            plt.subplot(10,5, midx+1)
            
            batch = it.next()
            image = batch[0].astype('uint8')
            plt.imshow(image)
            plt.axis('off')
        midx += 1
    
    plt.tight_layout()
    plt.show()

    
datagen = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.10,
    brightness_range=[0.6,1.4],
    channel_shift_range=0.7,
    width_shift_range=0.15,
    height_shift_range=0.15,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode='nearest'
) 
plot_augimages(np.random.choice(train_df['path'],10), datagen)

# Modelling

In [ ]:
y_count=len(train_df['classname'].unique())

## VGG19

In [ ]:
# include_top = False means that we doesnt include fully connected top layer we will add them accordingly
vgg19 = VGG19(include_top = False, input_shape = (560,560,3), weights = 'imagenet')

# training of all the convolution is set to false
for layer in vgg19.layers:
    layer.trainable = False

x = GlobalAveragePooling2D()(vgg19.output)
predictions = Dense(y_count, activation='softmax')(x)

model_vgg19 = Model(inputs = vgg19.input, outputs = predictions)

## ResNet50

In [ ]:
# include_top = False means that we doesnt include fully connected top layer we will add them accordingly
resNet50 = ResNet50(include_top = False, input_shape = (560,560,3), weights = 'imagenet')

# training of all the convolution is set to false
for layer in resNet50.layers:
    layer.trainable = False

x = GlobalAveragePooling2D()(resNet50.output)
predictions = Dense(y_count, activation='softmax')(x)

model_resNet50 = Model(inputs = resNet50.input, outputs = predictions)

## Own Proposed Model

In [ ]:
def create_model():
    model = Sequential()

    ## CNN 1
    model.add(Convolution2D(32,(3,3),activation='relu',input_shape=(64, 64, 3)))
    model.add(BatchNormalization())
    model.add(Convolution2D(32,(3,3),activation='relu',padding='same'))
    model.add(BatchNormalization(axis = 3))
    model.add(MaxPooling2D(pool_size=(2,2),padding='same'))
    model.add(Dropout(0.3))

    ## CNN 2
    model.add(Convolution2D(64,(3,3),activation='relu',padding='same'))
    model.add(BatchNormalization())
    model.add(Convolution2D(64,(3,3),activation='relu',padding='same'))
    model.add(BatchNormalization(axis = 3))
    model.add(MaxPooling2D(pool_size=(2,2),padding='same'))
    model.add(Dropout(0.3))

    ## CNN 3
    model.add(Convolution2D(128,(3,3),activation='relu',padding='same'))
    model.add(BatchNormalization())
    model.add(Convolution2D(128,(3,3),activation='relu',padding='same'))
    model.add(BatchNormalization(axis = 3))
    model.add(MaxPooling2D(pool_size=(2,2),padding='same'))
    model.add(Dropout(0.5))

    ## Output
    model.add(Flatten())
    model.add(Dense(512,activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(128,activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(10,activation='softmax'))

    return model


custom_model = create_model()

## Compile Model

### Vgg19

In [ ]:
model_vgg19.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
rlrp_vgg19 = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss",factor=0.01,patience=2,verbose=2,mode="auto",min_delta=0.0001,cooldown=0,min_lr=0)
model_vgg19.summary()

### ResNet50

In [ ]:
model_resNet50.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
rlrp_resNet50 = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss",factor=0.01,patience=2,verbose=2,mode="auto",min_delta=0.0001,cooldown=0,min_lr=0)
model_resNet50.summary()

### Custom Model



In [ ]:
custom_model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
custom_model.summary()

## Train and Test Split

In [ ]:
X, y = train_df[['path', 'classname']], train_df['classname']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Train Generators

### VGG19

In [ ]:
from keras.applications.vgg19 import preprocess_input

In [ ]:
train_generator_vgg_19 = datagen.flow_from_dataframe(
        X_train,  # This is the source directory for training images
        x_col='path',
        y_col='classname',
        target_size=(560, 560),  # All images will be resized to 150x150
        batch_size=32,
        class_mode="categorical",
        shuffle=True,
        preprocessing_function=preprocess_input
)

In [ ]:
val_generator_vgg_19 = datagen.flow_from_dataframe(
        X_test,  # This is the source directory for training images
        x_col='path',
        y_col='classname',
        target_size=(560, 560),  # All images will be resized to 150x150
        batch_size=32,
        class_mode="categorical",
        shuffle=True,
        preprocessing_function=preprocess_input
)

### ResNet50

In [ ]:
from tensorflow.keras.applications.resnet50 import preprocess_input

In [ ]:
train_generator_resnet50 = datagen.flow_from_dataframe(
        X_train,  # This is the source directory for training images
        x_col='path',
        y_col='classname',
        target_size=(560, 560),  # All images will be resized to 150x150
        batch_size=32,
        class_mode="categorical",
        shuffle=True,
        preprocessing_function=preprocess_input
)

In [ ]:
val_generator_resnet50 = datagen.flow_from_dataframe(
        X_test,  # This is the source directory for training images
        x_col='path',
        y_col='classname',
        target_size=(560, 560),  # All images will be resized to 150x150
        batch_size=32,
        class_mode="categorical",
        shuffle=True,
        preprocessing_function=preprocess_input
)

### Custom Model

In [ ]:
train_generator_custom_model = datagen.flow_from_dataframe(
        X_train,  # This is the source directory for training images
        x_col='path',
        y_col='classname',
        target_size=(64, 64),  # All images will be resized to 150x150
        batch_size=40,
        class_mode="categorical",
        shuffle=True,
)

In [ ]:
val_generator_custom_model = datagen.flow_from_dataframe(
        X_test,  # This is the source directory for training images
        x_col='path',
        y_col='classname',
        target_size=(64, 64),  # All images will be resized to 150x150
        batch_size=40,
        class_mode="categorical",
        shuffle=True,
)

## Model Fitting

### Vgg19

In [ ]:
history_vgg19 = model_vgg19.fit(
      train_generator_vgg_19,
     validation_data=val_generator_vgg_19,
      epochs=2,
      callbacks = [rlrp_vgg19],
      verbose=2)

### Resnet50

In [ ]:
history_resNet50 = model_resNet50.fit(
      train_generator_resnet50,
     validation_data=val_generator_resnet50,
      epochs=10,
       callbacks = [rlrp_resNet50], 
      verbose=2)

### Custom Model

In [ ]:
history_custom_model = custom_model.fit(
      train_generator_custom_model,
     validation_data=val_generator_custom_model,
      steps_per_epoch=100,
      epochs=70,
      verbose=2)

## Plot Loss

### Vgg19

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(history_vgg19.history['loss'])
plt.plot(history_vgg19.history['val_loss'])
plt.title('Model loss')
plt.ylabel('loss')
plt.xlabel('Epoch')
plt.show()

### ResNet50

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(history_resNet50.history['loss'])
plt.plot(history_resNet50.history['val_loss'])
plt.title('Model loss')
plt.ylabel('loss')
plt.xlabel('Epoch')
plt.show()

### Custom Model

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(history_custom_model.history['loss'])
plt.plot(history_custom_model.history['val_loss'])
plt.title('Model loss')
plt.ylabel('loss')
plt.xlabel('Epoch')
plt.show()

## Plot Accuracy

### Vgg19

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(history_vgg19.history['accuracy'])
plt.plot(history_vgg19.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.show()

### ResNet50

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(history_resNet50.history['accuracy'])
plt.plot(history_resNet50.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.show()

### Custom Model

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(history_custom_model.history['accuracy'])
plt.plot(history_custom_model.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.show()

# Explaniable AI

## Methods and utils

In [ ]:
dict_class = {'c0':0, 'c1':1, 'c2': 2, 'c3': 3, 'c4': 4, 'c5':5, 'c6': 6, 'c7':7, 'c8':8, 'c9':9}

In [ ]:
def gradcam_visualise(data, model, class_index):
    explainer = GradCAM()
    output = explainer.explain(data, model, class_index=class_index)
    return output

def activation_visualise(image, model, layers):
    explainer = ExtractActivations()
    output = explainer.explain([image], model, layers_name=layers)
    return output

In [ ]:
def plot_data_four(class_name, outputs):
    fig = plt.figure(figsize=(16, 16))
    plt.suptitle(classes[class_name])
    plt.subplot(2,2,1)
    plt.imshow(outputs[0])
    plt.subplot(2,2,2)
    plt.imshow(outputs[1])
    plt.subplot(2,2,3)
    plt.imshow(outputs[2])
    plt.subplot(2,2,4)
    plt.imshow(outputs[3])
    plt.show()

In [ ]:
def grad_cam(model, df_exp, class_name, class_index, image_size):
    output_data = []
    classes_df = df_exp[df_exp['classname'] ==  class_name].reset_index(drop = True)
    for idx,i in enumerate(np.random.choice(classes_df['path'],4)):
        image = cv2.imread(i)
        image = cv2.resize(image, image_size)
        data = ([image], None)
        output = gradcam_visualise(data, model, class_index)
        output_data.append(output)
    plot_data_four(class_name, output_data)

In [ ]:
def activations_model(model, df_exp, class_name, layers, image_size):
    output_data = []
    classes_df = df_exp[df_exp['classname'] ==  class_name].reset_index(drop = True)
    for idx,i in enumerate(np.random.choice(classes_df['path'],4)):
        image = cv2.imread(i)
        image = cv2.resize(image, image_size)
        image = tf.expand_dims(image, axis=0)
        output = activation_visualise([image], model, layers)
        output_data.append(output)
    plot_data_four(class_name, output_data)

## Resnet50

In [ ]:
for class_name in X_test['classname'].unique():
    grad_cam(model_resNet50, X_test, class_name, dict_class[class_name], (560,560))

In [ ]:
for class_name in X_test['classname'].unique():
    activations_model(model_resNet50, X_test, class_name, [model_resNet50.layers[-3].name], (560,560))

## VGG19

In [ ]:
for class_name in X_test['classname'].unique():
    grad_cam(model_vgg19, X_test, class_name, dict_class[class_name], (560,560))

In [ ]:
for class_name in X_test['classname'].unique():
    activations_model(model_vgg19, X_test, class_name, [model_vgg19.layers[-3].name], (560,560))

## Custom Model

In [ ]:
for class_name in X_test['classname'].unique():
    grad_cam(custom_model, X_test, class_name, dict_class[class_name], (64,64))

In [ ]:
for class_name in X_test['classname'].unique():
    activations_model(custom_model, X_test, class_name, [custom_model.layers[-9].name], (64,64))

## Model Save

In [ ]:
model_vgg19.save('./model_vgg19.h5')
model_resNet50.save('./model_resNet50.h5')
custom_model.save('./custom_model.h5')